In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
import sys
import pickle
from itertools import product

import numpy as np
import pandas as pd
import scipy.stats as stats

from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import seaborn as sns


sns.set_theme(style='whitegrid')
sns.set_theme(style='ticks')


In [ ]:
repo_dir = Path('../..')

In [ ]:
if str(repo_dir) not in sys.path:
    sys.path.append(str(repo_dir))
    
from analysis.curve_fitting.src.fitting_functions import LOSS_FUNCTIONS
from analysis.curve_fitting.src.utils import apply_filters, load_yaml, convert_loss_parameters, convert_loss_parameters_batch

from visualization.src.utils import COLOR_PALETTES, set_ticks, save_figs, COLORS
from visualization.src.visualize import plot_reg, plot_reg_bivariate, plot_confidence_intervals


In [ ]:
args = {
    'results_csv': repo_dir / 'results' / 'benchmark_scores.csv',
    'results_csv_ckpts': repo_dir / 'results' / 'benchmark_scores_ckpts.csv',
}

In [ ]:
results_csv = args['results_csv']
df_results = pd.read_csv(results_csv)

results_csv_ckpts = args['results_csv_ckpts']
df_results_ckpts = pd.read_csv(results_csv_ckpts)

## Experiment Configuration

## Apply Data Filters

In [ ]:
df_ckpts = df_results_ckpts[
    (df_results_ckpts['is_pretrained'] == False) &
    (df_results_ckpts['is_random'] == False) &
    (df_results_ckpts['is_ssl'] == False) &
    (df_results_ckpts['is_ablation'] == False)
    # (df_results_ckpts['is_adv'] == False)
]


In [ ]:
df_ckpts_avg = df_ckpts.groupby(['model_id', 'arch', 'dataset', 'arch_family', 'ckpt', 'n_samples_seen', 'n_params']).agg({'score':'mean'}).reset_index()

## Visualize

#### Plotting settings

In [ ]:
linewidth = 3.0
alpha_scatter = .5



color_palette = COLOR_PALETTES['models']
arch2colorid = {
    'ResNet': 5,
    'EfficientNet': 4,
    'ConvNeXt': 2,
    'ViT': 3
}

color_palette_regions = COLOR_PALETTES['regions']

In [ ]:


fig_multiplier = 0.7
figsize = (10, 6)
figsize = (fig_multiplier * figsize[0], fig_multiplier * figsize[1])

fig, axes = plt.subplots(1, 1, figsize=figsize, dpi=300)
ax = axes




data_plot = df_ckpts_avg.copy()
data_plot.rename(columns={'arch_family':'Architecture'}, inplace=True)

color_palette_ = [color_palette[arch2colorid[arch]] for arch in data_plot['Architecture'].unique()]
sns.scatterplot(data=data_plot, x='ckpt', y='score', style='Architecture', hue='Architecture', ax=ax, palette=color_palette_, s=10, alpha=alpha_scatter, legend=False)
sns.lineplot(data=data_plot, x='ckpt', y='score', style='Architecture', hue='Architecture', ax=ax, linewidth=linewidth, palette=color_palette_)
# sns.lineplot(data=data_plot, x='ckpt', y='score', style='arch', hue='Architecture', ax=ax, linewidth=linewidth, palette=color_palette_)

### Formatting
ax.set_xlabel('Training Epoch', fontsize=16, fontweight='bold')
ax.set_ylabel('Alignment Score (S)', fontsize=16, fontweight='bold')
ax.set_title('Alignment During Supervised Training', fontsize=20, fontweight='bold')
ax = set_ticks(ax, xticks_mode='log', yticks_mode=None, yticks=[0.0, 0.1, 0.2, 0.3, 0.4, 0.5])

handles, labels = ax.get_legend_handles_labels()
# h, l = handles[0], labels[0]
handles, labels = handles[:5], labels[:5]
ax.legend(handles, labels, loc='lower right')


ax.set_xscale('log')



ax.spines[['right', 'top']].set_visible(False)
plt.tight_layout()


figures_dir = '../figures'
fig_name = 'fig7_sup'
formats = ['pdf', 'png', 'svg']
save_figs(figures_dir, fig_name, formats=formats)

In [ ]:
figsize = (15, 9)
fig_multiplier = 1
figsize = (fig_multiplier * figsize[0], fig_multiplier * figsize[1])


fig, axes = plt.subplots(2, 2, figsize=figsize, dpi=300)
for arch_family, ax in zip(['ResNet', 'EfficientNet', 'ViT', 'ConvNeXt'], axes.flatten()):

    # ax = axes
    data_plot = df_ckpts.copy()
    
    data_plot = data_plot[data_plot.arch_family == arch_family]
    # data = data[data.dataset.isin(['imagenet', 'ecoset'])]

    data_plot.rename(columns={'arch_family':'Architecture'}, inplace=True)
    data_plot.rename(columns={'region':'Region'}, inplace=True)

    color = color_palette[arch2colorid[arch_family]]
    # sns.lineplot(data=data_plot, x='ckpt', y='score', style='Region', hue='Region', ax=ax, color=color, linewidth=linewidth, palette=color_palette_regions)
    sns.lineplot(data=data_plot, x='ckpt', y='score', style='Region', hue='Region', ax=ax, color=color, linewidth=linewidth)

    ax.set_xlabel('Training Epoch', fontsize=16, fontweight='bold')
    ax.set_ylabel('Alignment Score (S)', fontsize=16, fontweight='bold')
    ax.set_title(arch_family, fontsize=18, fontweight='bold')

    set_ticks(ax, xticks_mode='log', yticks_mode=None, yticks=[0.0, 0.1, 0.2, 0.3, 0.4, 0.5])
    handles, labels = ax.get_legend_handles_labels()
    # h, l = handles[0], labels[0]
    handles, labels = handles[:5], labels[:5]

    if arch_family != 'ConvNeXt':
        ax.legend().remove()

    ax.set_xscale('log')



    
ax.legend(bbox_to_anchor=(1.05, 1.05))
plt.tight_layout()

figures_dir = '../figures'
fig_name = 'fig10_sup_regions'
formats = ['pdf', 'png', 'svg']
save_figs(figures_dir, fig_name, formats=formats)
